In [1]:
#from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
#from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

#from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
#from MatrixFactorization.PureSVD import PureSVDRecommender

from Base.NonPersonalizedRecommender import TopPop, Random

#from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
#from GraphBased.RP3betaRecommender import RP3betaRecommender
#from GraphBased.P3alphaRecommender import P3alphaRecommender



In [2]:
import traceback, os
import scipy.sparse
URM_all = scipy.sparse.load_npz('URM_all_matrix.npz')
ICM_all = scipy.sparse.load_npz('ICM_all_matrix.npz')
URM_train = scipy.sparse.load_npz('URM_train_matrix.npz')
URM_test = scipy.sparse.load_npz('URM_test_matrix.npz')

In [4]:
recommender_list = [
    #Random,
    #TopPop,
    #P3alphaRecommender,
    #RP3betaRecommender,
    ItemKNNCFRecommender,
    #UserKNNCFRecommender,
    #MatrixFactorization_BPR_Cython,
   #MatrixFactorization_FunkSVD_Cython,
   #PureSVDRecommender,
    #SLIM_BPR_Cython,
    #SLIMElasticNetRecommender
    ]


from Base.Evaluation.Evaluator import SequentialEvaluator

evaluator = SequentialEvaluator(URM_test, [10], exclude_seen=True)


output_root_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_root_path):
    os.makedirs(output_root_path)


logFile = open(output_root_path + "result_all_algorithms.txt", "a")


for recommender_class in recommender_list:

    try:

        print("Algorithm: {}".format(recommender_class))


        recommender = recommender_class(URM_train)
        recommender.fit()
        results_run, results_run_string = evaluator.evaluateRecommender(recommender)
       
        print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_run_string))
        logFile.write("Algorithm: {}, results: \n{}\n".format(recommender.__class__, results_run_string))
        logFile.flush()


        
    except Exception as e:
        traceback.print_exc()
        logFile.write("Algorithm: {} - Exception: {}\n".format(recommender_class, str(e)))
        logFile.flush()



Algorithm: <class 'KNN.ItemKNNCFRecommender.ItemKNNCFRecommender'>
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 16600 ( 80 % ), 550.40 column/sec, elapsed time 0.50 min
Similarity column 20600 ( 100 % ), 550.66 column/sec, elapsed time 0.62 min
SequentialEvaluator: Processed 9676 ( 100.00% ) in 8.40 seconds. Users per second: 1152
Algorithm: <class 'KNN.ItemKNNCFRecommender.ItemKNNCFRecommender'>, results: 
CUTOFF: 10 - ROC_AUC: 0.2942410, PRECISION: 0.1757979, RECALL: 0.1740760, RECALL_TEST_LEN: 0.1757979, MAP: 0.0833598, MRR: 0.2279479, NDCG: 0.1429207, F1: 0.1749327, HIT_RATE: 0.7549607, ARHR: 0.2864341, NOVELTY: 0.0056869, DIVERSITY_MEAN_INTER_LIST: 0.9874448, DIVERSITY_HERFINDAHL: 0.9987343, COVERAGE_ITEM: 0.2877635, COVERAGE_USER: 0.1918091, DIVERSITY_GINI: 0.2373247, SHANNON_ENTROPY: 10.6938154, 



In [5]:
from createSubmission import create_submission

recommender=ItemKNNCFRecommender(URM_all)
recommender.fit()

create_submission(recommender)

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 16800 ( 81 % ), 558.78 column/sec, elapsed time 0.50 min
Similarity column 20600 ( 100 % ), 551.22 column/sec, elapsed time 0.62 min


TypeError: recommend() got an unexpected keyword argument 'n'